# Programmation fonctionnelle en Python : pour en savoir plus

Les éléments présentés ici ne sont pas au programme de la spécialité NSI au lycée. Ce sont des éléments de réflexion et de compréhension pour l'enseignant.

## Compétences pour la programmation fonctionnelle

L'activité du programmeur peut sembler assez différente en programmation impérative ou fonctionnelle. Effectivement, l'ordre dans lequel on appréhende les problèmes à résoudre diffère :
* en impératif, on doit penser la suite des ordres à appliquer pour progresser des données vers le résultat,
* en fonctionnel, on commence par écrire l'expression du résultat à calculer. 

Les compétences en jeu pour programmer de manière fonctionnelle sont cependant les mêmes qu'en programmation impérative:
* savoir **évaluer** la valeur d'une expression,
* savoir **anticiper** en écrivant le résultat attendu selon les valeurs des paramètres,
* savoir **décomposer** pour mettre en évidence une composition de fonctions,
* savoir **abstraire** en nommant un calcul par la fonction qui peut l'effectuer,
* savoir **généraliser** en identifiant un motif de récurrence connu.

## La récursivité

On distingue la récursivité **terminale** et **non terminale**.

Une fonction récursive est dite terminale si et seulement aucun appel récursif à la fonction en cours de définition n'est situé à l'intérieur d'une expression à évaluer *après* l'évaluation de l'appel récursif.

Elle est non terminale dans le cas opposé.

On peut constater le caractère terminal ou non, simplement à la lecture du texte de la fonction.

**Exemples** : la fonction `fact` ci-dessous est non-terminale. La fonction `modulo` ci-dessous est terminale. 

In [ ]:
def fact(n):
    return 1 if n==0 else n * fact(n-1)

fact(5)

In [ ]:
def modulo(a,b):
    return a if a < b else modulo (a-b, b)

modulo(42, 15)

**Remarque** : Pour observer le caractère terminal ou non d'une fonction récursive, on peut aussi utiliser un instrument adapté, par exemple l'environnement Thonny qui permet de visualiser en mode pas à pas, l'ensemble des appels récursifs par ouverture d'une nouvelle fenêtre à chaque appel d'une fonction.

Si le résultat final est calculé dans le dernier appel récursif et transmis en cascade aux appels précédents, la récursivité est terminale.

Si un résultat intermédiaire d'un appel sert d'argument à un calcul ultérieur, la récursivité est non terminale.

Cette distinction est utile pour évaluer la complexité en mémoire d'une fonction récursive. Dans le cas terminal, les contextes intermédiaires n'ont pas besoin d'être mémorisés. La récursion peut être traduite par une simple itération.

## Des schémas standards d'algorithmes récursifs

Le schéma suivant de **fonction récursive non terminale** applique une fonction de base `b` à `x`, si le prédicat `p` est vrai en `x`, sinon applique la fonction `f` avec le résultat de l'appel récursif sur la valeur suivante de `x`, calculée par `s`.

Ce schéma permet de généraliser la fonction `fact`.

In [ ]:
def frecnt (x, b, p, s, f):
    return b(x) if p(x) else f (x, frecnt(s(x), b, p, s, f))

frecnt(5, lambda x : 1, lambda x : x == 0, lambda x : x -1, lambda x, r : x * r)

Il permet aussi de calculer $2^n$...

In [ ]:
frecnt(10, lambda x : 1, lambda x : x == 0, lambda x : x -1, lambda x, r : 2 * r)

... ou la somme des n premiers entiers positifs.

In [ ]:
frecnt(10, lambda x : 0, lambda x : x == 0, lambda x : x -1, lambda x, r : x + r)

Le schéma suivant de **fonction récursive terminale**, applique une fonction de base `b` à `x`, si le prédicat `p` est vrai en `x`, sinon donne le résultat de l'appel récursif sur la valeur suivante de `x`, calculée par `s`.

Il permet de réécrire la fonction `modulo`...

In [ ]:
def frect (x, b, p, s):
    return b(x) if p(x) else frect(s(x), b, p, s)

frect((42,15), lambda x : x[0], lambda x : x[0] < x[1], lambda x : (x[0]-x[1], x[1]))

... et aussi une version terminale de la fonction `fact` en ajoutant un second paramètre pour accumuler la valeur du résultat.

In [ ]:
frect((10,1), lambda x : x[1], lambda x : x[0] == 0, lambda x : (x[0]-1, x[0]*x[1]))

**Activité** : utiliser un de ces deux schémas pour réécrire la fonction `pgcd`, la fonction `renverse`...

## De récursif en itératif

Transformer une fonction récursive en une fonction itérative équivalente est simple dans le cas de récursivité terminale. 

Il suffit d'itérer jusqu'au cas de base - pour lequel `p(x)` est vrai - en modifiant la valeur du paramètre pour obtenir la valeur suivante. Le résultat final est alors donné par la fonction `b`. 

On a ainsi un schéma itératif de calcul équivalent au schéma récursif terminal.

In [ ]:
def fiter (x, b, p, s):
    while not p(x):
        x = s(x)    
    return(b(x))

fiter ((10,1), lambda x : x[1], lambda x : x[0] == 0, lambda x : (x[0]-1, x[0]*x[1]))

**Activité** : tester ce schéma en l'utilisant  la place de `frect` dans l'activité précédente.

### Cas de la récursivité non terminale

Dérécursiver une fonction récursive non terminale est plus complexe, car après l'exécution de l'appel récursif `frecnt`, la valeur courante du paramètre `x`, est encore utilisée dans le calcul de `f`.

La solution consiste alors à simuler ce que fait l'interprète du langage lors de l'exécution de fonctions récursives, à savoir empiler les valeurs actuelles des paramètres avant appel pour les dépiler - et retrouver ainsi leurs valeurs - au moment du retour de l'appel récursif.

On utilise donc une pile appelée `contexte`, à laquelle on ajoute itérativement la valeur de `x` avant de passer à la suivante, jusqu'au cas de base. On peut alors dans la 2eme itération du programme, retrouver dans l'ordre en les dépilant, les valeurs successivement prises par `x` pour les utiliser dans les calculs successifs de `f`.

In [ ]:
def fiternt (x, b, p, s, f):
    contexte = []
    while not p(x):
        contexte.append (x)
        x = s(x)
    r = b(x)
    while contexte:
        x = contexte.pop ()
        r = f(x, r)
    return(r)

In [ ]:
fiternt(5, lambda x : 1, lambda x : x == 0, lambda x : x -1, lambda x, r : x * r)

**Activité** : tester ce schéma en l'utilisant à la place des usages de `frectnt` dans les activités précédentes.

## Des fonctions d'ordre supérieur
 
Les fonctions `map`, `filter` et `reduce` sont communes en programmation fonctionnelle et permettent de programmer des schémas de fonctions récursives sur les listes. Elle sont aussi présentes en Python.

* `map` permet d'appliquer la même fonction à tous les éléments d'une liste et de construire la liste des résultats.
* `filter` permet de tester un prédicat sur tous les éléments d'une liste et de construire la liste des éléments le vérifiant.
* `reduce` permet de généraliser un opérateur binaire associatif à tous les éléments d'une liste. Un paramètre optionnel permet de préciser la valeur à renvoyer pour une liste vide (l'élément neutre).


On peut ainsi calculer la liste des 10 premières puissances de 2.

In [ ]:
list(map(lambda n: 2**n, range(10)))

On peut filtrer une liste d'entiers pour ne conserver que les multiples de 3 ou de 5.

In [ ]:
list(filter(lambda n : n % 3 == 0 or n % 5 == 0, range(20)))

On peut généraliser le `+` et $\Sigma$ ou le `*` en $\Pi$.

In [ ]:
import functools

functools.reduce(lambda x, y: x + y, range(1,11))

In [ ]:
functools.reduce(lambda x, y: x * y, range(1,11))

### Une synthèse

Voici un exemple de programme combinant toutes ces fonctions. L'exemple est inspiré du [problème 1 du projet Euler](https://projecteuler.net/problem=1) :

> Si on liste tous les entiers naturels inférieurs à 10 qui sont multiples de 3 ou de 5, on obtient 3, 5, 6 and 9. La somme des carrés de ces multiples est 151.  
> Trouver la somme des carrés des multiples de 3 ou 5 inférieurs à 1000.

In [ ]:
functools.reduce(
    lambda m,n: m+n, 
    map(
        lambda n: n*n, 
        filter(
            lambda n: n%3==0 or n%5==0, 
            range(10)
        )
    )
)

**Activité** : Programmer des fonctions `mon_map(f, iterable)`, `mon_filter(f, iterable)`, `mon_reduce(f, iterable, initial=None)` ayant les mêmes effets que les fonctions présentées ici.

## Bilan

L'utilisation de fonctions en paramètres et la réflexion sur les schémas de fonctions permet de considérer des programmes comme des données et de construire des preuves. On a ainsi vu dans deux cas particuliers importants que tout programe récursif peut être traduit en programme impératif. Ce résultat est vrai de manière générale, attestant de l'équivalence en pouvoir d'expression de la programmation impérative et de la programmation fonctionnelle.

On ne s'en étonnera pas, si on se réfère aux théories fondatrices de l'informatique : les machines de Turing sont le modèle de machine permettant d'exécuter un programme impératif, le lambda-calcul est la théorie sur laquelle repose les langages fonctionnels. La thèse de Church énonce l'équivalence de ces deux modèles pour définir l'ensemble des fonctions calculables.


Equipe pédagoqique DIU EIL, ressource éducative libre distribuée sous [Licence Creative Commons Attribution - Pas d’Utilisation Commerciale - Partage dans les Mêmes Conditions 4.0 International](http://creativecommons.org/licenses/by-nc-sa/4.0/) ![Licence Creative Commons](https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png)